In [2]:
import os, sys
import pandas as pd
import numpy as np

def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")


base_path = os.path.join("data", "transformed")
gtr_persons = pd.read_csv(os.path.join(base_path, "persons.csv"))
gtr_orgs = pd.read_csv(os.path.join(base_path, "organisations.csv"))

Current working directory:  c:\Users\dec2g\GitHub\OrgSync


In [4]:
gtr_persons.head()

,person_id,EMPLOYED,PI_PER,COI_PER,PI_PER_ids,COI_PER_ids,EMPLOYED_ids,firstName,surname,otherNames,email,orcidId,created
0,0400AE80-674B-4068-A7D4-748DFA887CDE,['http://gtr.ukri.org/gtr/api/organisations/01...,['http://gtr.ukri.org/gtr/api/projects/1FF7C21...,['http://gtr.ukri.org/gtr/api/projects/A69BBDA...,"['1FF7C213-559F-4374-9932-069540778CFE', 'CF64...","['A69BBDA8-5CE1-41A2-872C-122079954D8F', '86E4...",['01F2924C-FFB4-481B-B8F0-31234D33F0FA'],Tom,Tregenza,NaN,NaN,0000-0003-4182-2222,1720391292000
1,0400FA13-4E69-4BB4-AACC-B8E13051441D,['http://gtr.ukri.org/gtr/api/organisations/AF...,['http://gtr.ukri.org/gtr/api/projects/F8DE3E2...,['http://gtr.ukri.org/gtr/api/projects/6734F47...,['F8DE3E25-99F4-4C8F-A5DE-511C98674535'],['6734F477-6EE2-4082-9765-ABB040C34AE1'],['AFE1E6E9-738F-4616-9D07-0781DAC34046'],Marianne,Yon,NaN,NaN,NaN,1720391292000
2,04F7BEB3-2C6E-4027-8B28-3CDB92DA0787,['http://gtr.ukri.org/gtr/api/organisations/6B...,['http://gtr.ukri.org/gtr/api/projects/B5879D1...,['http://gtr.ukri.org/gtr/api/projects/4163D0A...,"['B5879D1D-8A34-44B1-9EA0-08FC0B4DDB1A', '4D01...",['4163D0A1-31AE-40A7-89F5-1038BA805963'],['6BE9EFE8-67BC-4376-9142-6AE1A9AA6498'],Semira,Manaseki-Holland,NaN,NaN,NaN,1720391292000
3,04FC1BF4-0F97-4020-AEE6-0C4265BD96BE,['http://gtr.ukri.org/gtr/api/organisations/59...,['http://gtr.ukri.org/gtr/api/projects/BDEB021...,['http://gtr.ukri.org/gtr/api/projects/8C11A85...,['BDEB0211-F9D6-4F8B-80A1-5723764A22FB'],"['8C11A853-2407-4BD8-8063-48B8A2EC010E', '5D6A...",['595A5FEA-6A63-4445-BD0B-7CA0CC2EE7DF'],Julia,Edgar,NaN,NaN,NaN,1720391292000
4,05031B56-1BEB-4FE5-9FE3-ECA990789279,['http://gtr.ukri.org/gtr/api/organisations/7F...,['http://gtr.ukri.org/gtr/api/projects/80386E8...,['http://gtr.ukri.org/gtr/api/projects/65793E0...,"['80386E87-6008-484D-96D6-16751FBC1BCE', 'F9CF...","['65793E04-59E6-4C98-97C7-118FD0D97EFE', '1B2B...",['7FF630E0-3355-4076-9E47-528D4B0DBCB3'],Mark,Leake,NaN,NaN,NaN,1720391292000


# Fuzzy Matching Names
Taking the firstName, surname and othername columns, can we fuzzy match the names to identify if the same person has been entered multiple times with slight variations in the name?




In [ ]:
# Taking the firstName, surname and othername columns, can we fuzzy match the names to identify if the same person has been entered multiple times with slight variations in the name?
# We can use the fuzzywuzzy library to do this.
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Create a new column to store the full name of the person
gtr_persons["full_name"] = f"{gtr_persons['firstName']} {gtr_persons['otherNames']} {gtr_persons['surname']}" 

# we then want to fuzzy match all persons on "full_name" to find potential duplicates
# We will use the process.extract method to find the best matches for each person
# We will then filter the results to only include those that have a score above a certain threshold
# We will then manually inspect the results to determine if they are duplicates
# We will then merge the duplicates into a single record

# We will start by finding the best matches for each person
# We will use a threshold of 80 for now

threshold = 80
matches = gtr_persons["full_name"].apply(lambda x: process.extract(x, gtr_persons["full_name"], limit=10))
matches = matches.apply(lambda x: [y for y in x if y[1] > threshold])

# We will now manually inspect the results to determine if they are duplicates

# We will start by looking at the first 10 records

matches.head(10)



## PERSONS - ORGANISATION MATCHING
PERSONS:
```
person_id,EMPLOYED,PI_PER,COI_PER,PI_PER_ids,COI_PER_ids,EMPLOYED_ids,firstName,surname,otherNames,email,orcidId,created
0400AE80-674B-4068-A7D4-748DFA887CDE,['http://gtr.ukri.org/gtr/api/organisations/01F2924C-FFB4-481B-B8F0-31234D33F0FA'],"['http://gtr.ukri.org/gtr/api/projects/1FF7C213-559F-4374-9932-069540778CFE', 'http://gtr.ukri.org/gtr/api/projects/CF64FA13-BEAF-4A54-98E8-49CAF7EE0521', 'http://gtr.ukri.org/gtr/api/projects/071C84CD-586B-4D63-8A59-511470DE1DF1', 'http://gtr.ukri.org/gtr/api/projects/B834C500-C28E-4F8E-8B9D-5C22F03568B1', 'http://gtr.ukri.org/gtr/api/projects/9229A2EE-65B3-40C1-96FF-930012683DC3', 'http://gtr.ukri.org/gtr/api/projects/2C90145C-1F77-4CA8-BBF3-A4770F95712D', 'http://gtr.ukri.org/gtr/api/projects/56E94F19-C2BB-4317-B749-D9A6A67260EB', 'http://gtr.ukri.org/gtr/api/projects/CA0612B9-A3F9-4173-9A17-EAA8356BD011']","['http://gtr.ukri.org/gtr/api/projects/A69BBDA8-5CE1-41A2-872C-122079954D8F', 'http://gtr.ukri.org/gtr/api/projects/86E4C8C7-6F2F-4ECE-82E2-49FB247EEAA2']","['1FF7C213-559F-4374-9932-069540778CFE', 'CF64FA13-BEAF-4A54-98E8-49CAF7EE0521', '071C84CD-586B-4D63-8A59-511470DE1DF1', 'B834C500-C28E-4F8E-8B9D-5C22F03568B1', '9229A2EE-65B3-40C1-96FF-930012683DC3', '2C90145C-1F77-4CA8-BBF3-A4770F95712D', '56E94F19-C2BB-4317-B749-D9A6A67260EB', 'CA0612B9-A3F9-4173-9A17-EAA8356BD011']","['A69BBDA8-5CE1-41A2-872C-122079954D8F', '86E4C8C7-6F2F-4ECE-82E2-49FB247EEAA2']",['01F2924C-FFB4-481B-B8F0-31234D33F0FA'],Tom,Tregenza,,,0000-0003-4182-2222,1720391292000
0400FA13-4E69-4BB4-AACC-B8E13051441D,['http://gtr.ukri.org/gtr/api/organisations/AFE1E6E9-738F-4616-9D07-0781DAC34046'],['http://gtr.ukri.org/gtr/api/projects/F8DE3E25-99F4-4C8F-A5DE-511C98674535'],['http://gtr.ukri.org/gtr/api/projects/6734F477-6EE2-4082-9765-ABB040C34AE1'],['F8DE3E25-99F4-4C8F-A5DE-511C98674535'],['6734F477-6EE2-4082-9765-ABB040C34AE1'],['AFE1E6E9-738F-4616-9D07-0781DAC34046'],Marianne,Yon,,,,1720391292000
04F7BEB3-2C6E-4027-8B28-3CDB92DA0787,['http://gtr.ukri.org/gtr/api/organisations/6BE9EFE8-67BC-4376-9142-6AE1A9AA6498'],"['http://gtr.ukri.org/gtr/api/projects/B5879D1D-8A34-44B1-9EA0-08FC0B4DDB1A', 'http://gtr.ukri.org/gtr/api/projects/4D010246-3D04-4A07-AE9A-42B6CAF438DC']",['http://gtr.ukri.org/gtr/api/projects/4163D0A1-31AE-40A7-89F5-1038BA805963'],"['B5879D1D-8A34-44B1-9EA0-08FC0B4DDB1A', '4D010246-3D04-4A07-AE9A-42B6CAF438DC']",['4163D0A1-31AE-40A7-89F5-1038BA805963'],['6BE9EFE8-67BC-4376-9142-6AE1A9AA6498'],Semira,Manaseki-Holland,,,,1720391292000
```


ORGANISATIONS:
```
```

### Emplyees
Organisations will have a list of employees. Employee links will be in two columns, EMPLOYEE and EMPLOYEE_ids. 
Employees from organisations should be joined with persons "EMPLOYED_ids" column

### Projects
For projects, both organisations and persons will be linked via the href directly, since they both refer to a third domain.

Organisations will have "PROJECTS" column. Persons will have "PI_PER" and "COI_PER" columns indicating project hrefs they are PI of, or colaborator of. 

### Matching
We need to find in each case any different organisations that have any employees, pis on projects, or cois on projects in common.  

